In [24]:
import numpy as np 
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [25]:
import json
import pandas as pd

def convert(x):
    ob = json.loads(x)
    for k, v in ob.items():
        if isinstance(v, list):
            ob[k] = ','.join(v)
        elif isinstance(v, dict):
            for kk, vv in v.items():
                ob['%s_%s' % (k, kk)] = vv
            del ob[k]
    return ob

In [26]:
json_filename='review_sample_cleveland.json'
with open(json_filename,'rb') as f:
    data = f.readlines()

df = pd.DataFrame([convert(line) for line in data])

In [27]:
del(data)

In [28]:
data = df[['text', 'stars']]

In [29]:
data['sentiment']=['pos' if (x>3) else 'neg' for x in data['stars']]

C:\Users\pulki\Anaconda3\envs\tensorflow_env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [30]:
data['text']= [x.lower() for x in data['text']]
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

C:\Users\pulki\Anaconda3\envs\tensorflow_env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\pulki\Anaconda3\envs\tensorflow_env\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [31]:

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')

In [32]:
pd.set_option('display.max_colwidth',-1)
data[:5]

,text,stars,sentiment
0,dipped cone all the way this location is tricky because it gets very busy in the evenings best route dont go in the drivethru park your car and order inside its quicker and easier,3,neg
1,always amazing food and service love their chicken parm sandwich their blizzards are awesome too,5,pos
2,this place is literally the worst hopefully the got rid of the awful management because the owner is actually sort of nice i ordered ice cream and just got a sloppy mess and no apology,1,neg
3,very nice and new staff is always friendly but the drive thru is ridiculous two burgers and a diet soda and im still sitting here 10 minutes after the fact people have stuff to do i could cook a meal at home in this time,3,neg
4,nothing special about this place their ice cream is ok on busy nights drive thru is insane and impossible to turn left out of here prices are high too the employees are really nice and it is a clean store,3,neg


In [33]:
data.dtypes

text         object
stars        int64 
sentiment    object
dtype: object

In [34]:
data['text']= [x.encode('ascii') for x in data['text']]

UnicodeEncodeError: 'ascii' codec can't encode characters in position 35-36: ordinal not in range(128)

In [35]:
tokenizer = Tokenizer(nb_words=2000, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',lower=True,split=' ')
tokenizer.fit_on_texts(data['text'].values)

In [36]:
tokenizer

In [37]:
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

In [38]:
len(data)

91057

In [39]:
embed_dim = 128
lstm_out = 196
batch_size = 32

model = Sequential()
model.add(Embedding(2000, embed_dim,input_length = X.shape[1], dropout = 0.2))
model.add(LSTM(lstm_out, dropout_U = 0.2, dropout_W = 0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

C:\Users\pulki\Anaconda3\envs\tensorflow_env\lib\site-packages\ipykernel_launcher.py:6: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  


Instructions for updating:
Colocations handled automatically by placer.


C:\Users\pulki\Anaconda3\envs\tensorflow_env\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(196, dropout=0.2, recurrent_dropout=0.2)`
  import sys


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 901, 128)          256000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [40]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(72845, 901) (72845, 2)
(18212, 901) (18212, 2)


In [41]:
model.fit(X_train, Y_train, batch_size = batch_size, nb_epoch = 1, verbose = 2)

Instructions for updating:
Use tf.cast instead.


C:\Users\pulki\Anaconda3\envs\tensorflow_env\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/1
 - 3466s - loss: 0.3798 - acc: 0.8385


In [42]:
model

In [43]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.2f" % (acc))

Score: 0.30
Validation Accuracy: 0.88
